In [1]:
import pandas as pd
import numpy as np

In [ ]:
prediction_df = {}
for sample_name in tqdm.tqdm(sample_names):
    df = pd.read_csv(f'/home/compomics/conode53_pride/PRIDE_DATA/PXD007592/MS2RESCORE_v3.2.0-alpha-aee8ec7-b1/{sample_name}/results.sage.ms2rescore.mokapot.peptides.txt', sep='\t')
    df = df[(df['mokapot q-value']<=0.01)&(df[df['is_target'] == True])]
    #drop rows where proteins contains the substring "rev"
    df = df[~df['protein_list'].str.contains('CONTAMINANT', na=False)]
    df = df[~df['protein_list'].str.contains('rev', na=False)]
    dfquant = df[['proteins', 'ms2_intensity']]
    #split and explode proteins on ';'
    dfquant['proteins'] = dfquant['proteins'].str.split(';')
    dfquant = dfquant.explode('proteins')

    #proteins split on | and extract second element
    dfquant['proteins'] = dfquant['proteins'].str.split('|').str[1]
    #minmax normalize ms2_intensity
    dfquant['ms2_intensity'] = (dfquant['ms2_intensity'] - dfquant['ms2_intensity'].min()) / (dfquant['ms2_intensity'].max() - dfquant['ms2_intensity'].min())
    test = mlmarker.MLMarker(dfquant.set_index('proteins').T, binary=False, dev=True)
    prediction = test.explainability.adjusted_absent_shap_values_df(n_preds=100).T.sum(axis=0)
    prediction_df[sample_name]= prediction
prediction_df = pd.DataFrame(prediction_df)
prediction_df

In [7]:
import pandas as pd
import glob

file_list = glob.glob('/home/compomics/conode53_pride/PRIDE_DATA/PXD007592/IONBOT_v0.11.3/*/flashlfq.tsv')

# Create an empty DataFrame to store the combined data
combined_df = pd.DataFrame()

# Iterate over each file and append the data to the combined DataFrame
for file in file_list:
    # Read the data from the file
    df = pd.read_csv(file, sep='\t')
    
    # Add a new column called "filename" with the file name
    df['filename'] = file
    
    # Append the data to the combined DataFrame
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Print the combined DataFrame
combined_df.head()


,File Name,Base Sequence,Precursor Charge,Scan Retention Time,Full Sequence,Peptide Monoisotopic Mass,Protein Accession,filename
0,poor_responder_12_1.mgf,YLIANATNPESK,2,26.41917,YLIANATNPESK,1319.67213,1433T_HUMAN,/home/compomics/conode53_pride/PRIDE_DATA/PXD0...
1,poor_responder_12_1.mgf,DVTPEATKEVIER,2,26.42578,DVTPEATKEVIER,1485.76749,GEPH_HUMAN,/home/compomics/conode53_pride/PRIDE_DATA/PXD0...
2,poor_responder_12_1.mgf,HSIEVPIPR,2,26.43552,HSIEVPIPR,1046.58728,CNTN1_HUMAN,/home/compomics/conode53_pride/PRIDE_DATA/PXD0...
3,poor_responder_12_1.mgf,HIYLLPSGR,2,26.43741,HIYLLPSGR,1054.59236,AATC_HUMAN,/home/compomics/conode53_pride/PRIDE_DATA/PXD0...
4,poor_responder_12_1.mgf,SIAEEQYSDLEKEK,2,26.44981,SIAEEQYSDLEKEK,1667.78901,ROCK2_HUMAN,/home/compomics/conode53_pride/PRIDE_DATA/PXD0...


In [ ]:
combined_df.pivot_table(index=['Base Sequence', 'Protein Accession'], columns='File Name', values='')

In [2]:
df = pd.read_csv('/home/compomics/conode53_pride/PRIDE_DATA/PXD007592/QuantifiedPeaks_v2.tsv', sep='\t')
df.head()

/tmp/ipykernel_67400/3185968166.py:1: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/home/compomics/conode53_pride/PRIDE_DATA/PXD007592/QuantifiedPeaks_v2.tsv', sep='\t')


,File Name,Base Sequence,Full Sequence,Protein Group,Peptide Monoisotopic Mass,MS2 Retention Time,Precursor Charge,Theoretical MZ,Peak intensity,Peak RT Start,...,Peak Charge,Num Charge States Observed,Peak Detection Type,MBR Score,PSMs Mapped,Base Sequences Mapped,Full Sequences Mapped,Peak Split Valley RT,Peak Apex Mass Error (ppm),Unnamed: 22
0,poor_responder_3_1,CIMMF,CIMMF,poor_responder_3_1: controllerType=0 controlle...,999.543800,57.270275,2,500.779176,2.882076e+11,46.842321,...,2,1,MSMS,NaN,1,1,1,0.000000,-13.803567,NaN
1,poor_responder_3_1,CFCLI|CFFAA|CFIHD|CGAEG|CGDMA|CGGMI|CGLHI|CHCI...,CFCLI|CFFAA|CFIHD|CGAEG|CGDMA|CGGMI|CGLHI|CHCI...,poor_responder_3_1: controllerType=0 controlle...,999.543762,46.941818,2,500.779158,2.759118e+11,46.842321,...,2,1,MSMS,NaN,17,17,17,0.000000,-11.445390,NaN
2,poor_responder_3_1,DACFM|DAEHI|DAHCL|DALLL|DCIFI,DACFM|DAEHI|DAHCL|DALLL|DCIFI,poor_responder_3_1: controllerType=0 controlle...,1459.732727,62.930487,3,487.584852,1.495602e+11,61.703213,...,3,1,MSMS,NaN,5,5,5,68.006179,-13.010117,NaN
3,poor_responder_3_1,CEDDG|CEFID|CEIGG|CFACE|CFIMG|CGDDH|CHCEF|CILFD,CEDDG|CEFID|CEIGG|CFACE|CFIMG|CGDDH|CHCEF|CILFD,poor_responder_3_1: controllerType=0 controlle...,1313.670166,44.208197,2,657.842359,6.450113e+10,44.157707,...,2,1,MSMS,NaN,8,8,8,0.000000,-6.710701,NaN
4,poor_responder_3_1,DCDCA,DCDCA,poor_responder_3_1: controllerType=0 controlle...,1953.081000,66.019915,3,652.034276,6.375635e+10,64.469146,...,3,1,MSMS,NaN,1,1,1,0.000000,-11.773918,NaN


In [4]:
#see all pandas columns
pd.set_option('display.max_columns', None)
#set them as wide as possible
pd.set_option('display.width', 1000)
df.head()

,File Name,Base Sequence,Full Sequence,Protein Group,Peptide Monoisotopic Mass,MS2 Retention Time,Precursor Charge,Theoretical MZ,Peak intensity,Peak RT Start,Peak RT Apex,Peak RT End,Peak MZ,Peak Charge,Num Charge States Observed,Peak Detection Type,MBR Score,PSMs Mapped,Base Sequences Mapped,Full Sequences Mapped,Peak Split Valley RT,Peak Apex Mass Error (ppm),Unnamed: 22
0,poor_responder_3_1,CIMMF,CIMMF,poor_responder_3_1: controllerType=0 controlle...,999.543800,57.270275,2,500.779176,2.882076e+11,46.842321,47.31179,67.092294,500.772278,2,1,MSMS,NaN,1,1,1,0.000000,-13.803567,NaN
1,poor_responder_3_1,CFCLI|CFFAA|CFIHD|CGAEG|CGDMA|CGGMI|CGLHI|CHCI...,CFCLI|CFFAA|CFIHD|CGAEG|CGDMA|CGGMI|CGLHI|CHCI...,poor_responder_3_1: controllerType=0 controlle...,999.543762,46.941818,2,500.779158,2.759118e+11,46.842321,47.335898,70.993706,500.772247,2,1,MSMS,NaN,17,17,17,0.000000,-11.445390,NaN
2,poor_responder_3_1,DACFM|DAEHI|DAHCL|DALLL|DCIFI,DACFM|DAEHI|DAHCL|DALLL|DCIFI,poor_responder_3_1: controllerType=0 controlle...,1459.732727,62.930487,3,487.584852,1.495602e+11,61.703213,63.412208,67.996339,487.57782,3,1,MSMS,NaN,5,5,5,68.006179,-13.010117,NaN
3,poor_responder_3_1,CEDDG|CEFID|CEIGG|CFACE|CFIMG|CGDDH|CHCEF|CILFD,CEDDG|CEFID|CEIGG|CFACE|CFIMG|CGDDH|CHCEF|CILFD,poor_responder_3_1: controllerType=0 controlle...,1313.670166,44.208197,2,657.842359,6.450113e+10,44.157707,44.572929,57.190075,657.836548,2,1,MSMS,NaN,8,8,8,0.000000,-6.710701,NaN
4,poor_responder_3_1,DCDCA,DCDCA,poor_responder_3_1: controllerType=0 controlle...,1953.081000,66.019915,3,652.034276,6.375635e+10,64.469146,65.035305,76.01809,652.026611,3,1,MSMS,NaN,1,1,1,0.000000,-11.773918,NaN


In [5]:
df['Protein Group'].values

array(['poor_responder_3_1: controllerType=0 controllerNumber=1 scan=26069',
       'poor_responder_3_1: controllerType=0 controllerNumber=1 scan=20980;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=21324;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=21806;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=22178;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=22532;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=22923;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=23293;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=23678;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=24048;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=24427;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=24891;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=25633;poor_responder_3_1: controllerType=0 controllerNumber=1 scan=26450;poor_responder_3_1: controllerType=0 control

In [3]:
df.columns

Index(['File Name', 'Base Sequence', 'Full Sequence', 'Protein Group',
       'Peptide Monoisotopic Mass', 'MS2 Retention Time', 'Precursor Charge',
       'Theoretical MZ', 'Peak intensity', 'Peak RT Start', 'Peak RT Apex',
       'Peak RT End', 'Peak MZ', 'Peak Charge', 'Num Charge States Observed',
       'Peak Detection Type', 'MBR Score', 'PSMs Mapped',
       'Base Sequences Mapped', 'Full Sequences Mapped',
       'Peak Split Valley RT', 'Peak Apex Mass Error (ppm)', 'Unnamed: 22'],
      dtype='object')

In [ ]:
subset = df[['']]